# Conversation Patterns

In the previous chapter we used two-agent conversation, which
can be started by the `initiate_chat` method.
Two-agent chat is a useful conversation pattern but AutoGen offers more.
In this chapter, we will first dig a little bit more into the two-agent 
chat pattern and chat result, 
then we will show you several conversation patterns that involve 
more than two agents.


## Two-Agent Chat and Chat Result

Two-agent chat is the simplest from of conversation pattern. 
We start a two-agent chat using the `initiate_chat` method of every 
`ConversableAgent` agent.
We have already
seen multiple examples of two-agent chats in previous chapters
but we haven't covered the details.

The following figure illustrates how two-agent chat works.

```{=mdx}
![Two-agent chat](./assets/two-agent-chat.png)
```

A two-agent chats takes two inputs: a message, which is a string provided by
the caller; a context, which specifies various parameters of the chat.
The sender agent uses its chat initializer method
(i.e., `generate_init_message` method of `ConversableAgent`)
to generate an initial message from the inputs, and
sends it to the recipient agent to start the chat.
The sender agent is the agent whose `initiate_chat` method is called, and
the recipient agent is the other agent.

Once the chat terminates, the history of the chat is processed by a 
chat summarizer. The summarizer summarizes the chat history and calculates
the token usage of the chat. 
You can configure the type of summary using the `summary_method`
parameter of the `initiate_chat` method. By default, it is the last
message of the chat (i.e., `summary_method='last_msg'`).

The example below is a two-agent chat between a student agent and a teacher
agent. Its summarizer uses an LLM-based summary.

In [2]:
import os
from autogen import ConversableAgent

student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)
teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are a math teacher.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="What is triangle inequality?",
    summary_method="reflection_with_llm",
    max_turns=2,
)

Student_Agent (to Teacher_Agent):

What is triangle inequality?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Teacher_Agent (to Student_Agent):

Triangle inequality theorem is a fundamental principle in geometry that states that the sum of the lengths of any two sides of a triangle must always be greater than the length of the third side. In a triangle with sides of lengths a, b, and c, the theorem can be written as:

a + b > c
a + c > b
b + c > a

Each of these represents the condition for one specific side (a, b, or c). All must be true for a triangle to exist.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Student_Agent (to Teacher_Agent):

Thank you for the explanation. This theorem helps in understanding the basic properties of a triangle. It can also be useful when solving geometric problems or proving other mathematical theorems. Can you give me an e

Let's see what the summary looks like.
The summary is stored in the `chat_result` object of the type `ChatResult` 
that was returned by the `initiate_chat` method.

In [15]:
print(chat_result.summary)

The triangle inequality theorem states that in a triangle, the sum of the lengths of any two sides must always be greater than the length of the third side. This principle is significant in geometry and is used in solving problems or proving theorems. For instance, if given three line segments, you can determine if they can form a triangle using this theorem.


In the above example, the summary method is set to `reflection_with_llm` which
takes a list of messages from the conversation and summarize them 
using a call to an LLM.
The summary method first tries to use the recipient's LLM, if it is not available
then it uses the sender's LLM. In this case the recipient is "Teacher_Agent" and
the sender is "Student_Agent".
The input prompt for the LLM is the following default prompt:

In [13]:
print(ConversableAgent.DEFAULT_summary_prompt)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


You can also use a custom prompt by setting the `summary_prompt` argument
of `initiate_chat`.

There are some other useful information in the `ChatResult` object, including
the conversation history, human input, and token cost.

In [16]:
# Get the chat history.
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': 'What is triangle inequality?', 'role': 'assistant'},
 {'content': 'Triangle inequality theorem is a fundamental principle in '
             'geometry that states that the sum of the lengths of any two '
             'sides of a triangle must always be greater than the length of '
             'the third side. In a triangle with sides of lengths a, b, and c, '
             'the theorem can be written as:\n'
             '\n'
             'a + b > c\n'
             'a + c > b\n'
             'b + c > a\n'
             '\n'
             'Each of these represents the condition for one specific side (a, '
             'b, or c). All must be true for a triangle to exist.',
  'role': 'user'},
 {'content': 'Thank you for the explanation. This theorem helps in '
             'understanding the basic properties of a triangle. It can also be '
             'useful when solving geometric problems or proving other '
             'mathematical theorems. Can you give me an example of ho

That chat messages in the chat result are from the 
recipient agent's perspective -- the sender is the "assistant" and the recipient
is the "user".

In [8]:
# Get the cost of the chat.
pprint.pprint(chat_result.cost)

({'gpt-4-0613': {'completion_tokens': 399,
                 'cost': 0.04521,
                 'prompt_tokens': 709,
                 'total_tokens': 1108},
  'total_cost': 0.04521},
 {'total_cost': 0})


## Sequential Chats

The name of this pattern is self-explanatory -- it is a sequence of
chats between two agents, chained together by a mechanism called *carryover*,
which brings the summary of the previous chat to the context
of the next chat.

This pattern is useful for complex task that can be broken down into 
interdependent sub-tasks. 
The figure below illustrate how this pattern works.

```{=mdx}
![initiate_chats](./assets/sequential-two-agent-chat.png)
```

In this pattern, the a pair of agents first start a two-agent chat,
then the summary of the conversation becomes a *carryover* for the next
two-agent chat. The next chat passes the carryover to the `carryover` parameter
of the context to generate its initial message.

Carryover accumulates as the conversation moves forward,,
so each subsequent chat starts with all the carryovers 
from previous chats.

The figure above shows distinct recipient agents for all the chats, however,
the recipient agents in the sequence are allowed to repeat.

To illustrate this pattern, let's consider a simple example of arithmetic
operator agents. One agent 
(called the "Number_Agent") is responsible for coming up with a number,
and other agents are responsible for performing a specific arithmetic
operation on the number, e.g., add 1, multiply by 2, etc..

In [17]:
# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

The Number Agent chats with the first operator agent, then the second
operator agent, and so on.
After each chat, the last message in the conversation (i.e., the result of the arithmetic operation from the operator agent)
is used as the summary of the chat.
This is specified by the `summary_method` parameter.
In the end we will have the result of the arithmetic operations.

In [9]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
14

With the following carryover: 


********************************************************************************
Number_Agent (to Adder_Agent):

14

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_Agent):

15

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

16

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
These are my numbers

With the following carryover: 
16

********************************************************************************
Number_Agent (to Mul

First thing to note is that the `initiate_chats` method takes a list of
dictionaries, each dictionary contains the arguments for the `initiate_chat`
method.

Second, each chat in the sequence has a maximum round of 2, as specified with
the setting `max_turns=2`, 
which means each arithmetic operation is performed twice.
So you can see in the first chat the number 14 becomes 15 and then 16,
in the second chat the number 16 becomes 32 and then 64, and so on.

Third, the carryover accumulates as the chats go on. In the second chat,
the carryover is the summary of the first chat "16".
In the third chat, the carryover is the summary of the first and second
chat, which is the list "16" and "64", and both numbers are operated upon.
In the forth and last chat, the carryover is the summary of all previous
chats, which is the list "16", "64", "14" and "62", and all of these
numbers are operated upon.

The final note is that the `initiate_chats` method returns a list of
`ChatResult` objects, one for each chat in the sequence.

In [10]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  16
Second Chat Summary:  64
Third Chat Summary:  14
62
Fourth Chat Summary:  4
16
3.5
15.5


Besides calling `initiate_chats` from the same sender agent, you can also
call a high-level function `autogen.agentchat.initiate_chats` to start
a sequence of two-agent chats with different sender agents. 
This function allows you to specify the
sender agent for each chat in the sequence.

## Group Chat

So far we have only seen conversation patterns that involve two agents or 
a sequence of two-agent chats. AutoGen provides a more general conversation
pattern called group chat, which involves more than two agents.
The core idea of group chat is that all agents contribute to a single
conversation thread and share the same context.
This is useful for tasks that require collaboration among multiple agents.

The figure below illustrates how group chat works.

```{=mdx}
![group_chat](./assets/group-chat.png)
```

A group chat is orchestrated by a special
agent type `GroupChatManager`. In the first step of the group chat,
the Group Chat Manager selects an agent to speak. Then, the selected agent
speaks and the message is sent back to the Group Chat Manager,
who **broadcasts** the message to all other agents in the group.
This process repeats until the conversation stops.

The Group Chat Manager can use several strategies to select the next agent.
Currently, the following strategies are supported:

1. `round_robin`: The Group Chat Manager selects agents in a round-robin
   fashion based on the order of the agents provided.
2. `random`: The Group Chat Manager selects agents randomly.
3. `manual`: The Group Chat Manager selects agents by asking for human input.
4. `auto`: The default strategy, which selects agents using the Group Chat
   Manager's LLM.

To illustrate this pattern, let's consider a simple example of a group chat
among the same arithmetic operator agents as in the previous example,
with the objective of turning a number into a specific target number
using a sequence of arithmetic operations powered by the agents.

In this example, we use the `auto` strategy to select the next agent.
To help the Group Chat Manager select the next agent, we also set the
`description` of the agents. Without the `description`, the Group Chat Manager
will use the agents' `system_message`, which may be not be the best choice.

In [11]:
# The `description` attribute is a string that describes the agent.
# It can also be set in `ConversableAgent` constructor.
adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."

We first create a `GroupChat` object and provide the list of agents.
If we were to use the `round_robin` strategy, this list would specify the order
of the agents to be selected.
We also initialize the group chat with an empty message list and a maximum
round of 6, which means there will be at most 6 iteratiosn of selecting speaker,
agent speaks and broadcasting message.

In [12]:
from autogen import GroupChat

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
)

Now we create a `GroupChatManager` object and provide the `GroupChat` object as
input. We also need to specify the `llm_config` of the Group Chat Manager
so it can use the LLM to select the next agent (the `auto` strategy).

In [13]:
from autogen import GroupChatManager

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

Finally, we have the Number Agent from before to start a two-agent chat with
the Group Chat Manager, which runs the group chat internally and terminates
the two-agent chat when the internal group chat is done.
Because the Number Agent is selected to speak by us, it counts as the 
first round of the group chat.

In [14]:
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

7

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

14

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

13

--------------------------------------------------------------------------------
Number_Agent (to chat_manager):

13

--------------------------------------------------------------------------------


You can see that the Number Agent is selected to speak first, then the
Group Chat Manager selects the Multiplier Agent to speak, then the Adder
Agent, and so on. The number is operated upon by each agent in the group
chat, and the final result is 13.

We can take a look at the summary of the group chat, provided by the 
`ChatResult` object returned by the `initiate_chat` method.

In [15]:
print(chat_result.summary)

The agents cooperatively manipulated the initial number (3) through multipliying, adding, and subtracting operations to reach the target number (13).


### Send Introductions

In the previous example, we set the `description` of the agents to help the
Group Chat Manager select the next agent. This only helps the Group Chat Manager,
however, does not help the participating agents to know about each other.
Sometimes it is useful have each agent introduce themselves to other
agents in the group chat. This can be done by setting the `send_introductions=True`.

In [16]:
group_chat_with_introductions = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
    send_introductions=True,
)

Under the hood, the Group Chat Manager sends a message containing the 
agents' names and descriptions to all agents in the group chat before the
group chat starts.

### Group Chat in a Sequential Chat

Group chat can also be used as a part of a sequential chat.
In this case, the Group Chat Manager is treated as a regular agent
in the sequence of two-agent chats.

In [17]:
# Let's use the group chat with introduction messages created above.
group_chat_manager_with_intros = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

# Start a sequence of two-agent chats between the number agent and
# the group chat manager.
chat_result = number_agent.initiate_chats(
    [
        {
            "recipient": group_chat_manager_with_intros,
            "message": "My number is 3, I want to turn it into 13.",
        },
        {
            "recipient": group_chat_manager_with_intros,
            "message": "Turn this number to 32.",
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
My number is 3, I want to turn it into 13.

With the following carryover: 


********************************************************************************
Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

7

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

14

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

13

--------------------------------------------------------------------------------
Number_Agent (to chat_manager):

Your number is 13.

----------------------

/Users/ekzhu/autogen/autogen/agentchat/chat.py:46: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


In the above example, the Group Chat Manager runs the group chat two times.
In the first time the number 3 becomes 13, and the last message of this group chat
is being used as the carryover for the next group chat, which starts from 13.

You can also see from the warning message that the Group Chat Manager's
history is being cleared after the first group chat, which is the default.
To keep the history of the Group Chat Manager, 
you can set the `clear_history=False` for the first chat.

### Constrained Speaker Selection

Group chat is a powerful conversation pattern, but it can be hard to control
if the number of participating agents is large.
AutoGen provides a way to constrain the selection of the next speaker
by using the `allowed_or_disallowed_speaker_transitions` argument of the 
`GroupChat` class.

The `allowed_or_disallowed_speaker_transitions` argument is a dictionary
that maps a given agent to a list of agents that can (or cannot) 
be selected to speak next. The `speaker_transitions_type` argument specifies
whether the transitions are allowed or disallowed.

Here is an example:

In [18]:
allowed_transitions = {
    number_agent: [adder_agent, number_agent],
    adder_agent: [multiplier_agent, number_agent],
    subtracter_agent: [divider_agent, number_agent],
    multiplier_agent: [subtracter_agent, number_agent],
    divider_agent: [adder_agent, number_agent],
}

In this example, the allowed transitions are specified for each agent.
The Number Agent can be followed by the Adder Agent and the Number Agent, 
the Adder Agent can be followed by the Multiplier Agent and the Number Agent,
and so on.
Let's put this into the group chat and see how it works.
The `speaker_transitions_type` is set to `allowed` so the transitions are
positive constraints.

In [19]:
constrained_graph_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
    messages=[],
    max_round=12,
    send_introductions=True,
)

constrained_group_chat_manager = GroupChatManager(
    groupchat=constrained_graph_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

chat_result = number_agent.initiate_chat(
    constrained_group_chat_manager,
    message="My number is 3, I want to turn it into 10. Once I get to 10, keep it there.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 10. Once I get to 10, keep it there.

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

8

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

7

--------------------------------------------------------------------------------
Divider_Agent (to chat_manager):

3.5

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

4.5

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

9

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

8

----------------------------------------------------

This time, the agents are selected following the constraints we have specified.

## Nested Chats

The previous conversations patterns (two-agent chat, sequential chat, and group chat)
are useful for building complex workflows, however, they do not expose a
single conversational interface, which is often needed for scenarios like
question-answering bots and personal assistants.
In some other cases, it is also useful to package a workflow into a single
agent for reuse in a larger workflow. 
AutoGen provides a way to achieve this by using nested chats.

Nested chats is powered by the nested chats handler, which is a pluggable
component of `ConversableAgent`.
The figure below illustrates how the nested chats handler triggers
a sequence of nested chats when a message is received.

```{=mdx}
![nested_chat](./assets/nested-chats.png)
```

When a message comes in and passes the [human-in-the-loop component](./human-in-the-loop), 
the nested chats handler checks if the message should trigger a nested chat
based on conditions specified by the user.
If the conditions are met, the nested chats handler starts a sequence of
nested chats specified using the sequential chats pattern.
In each of the nested chats, the sender agent is always the same agent
that triggered the nested chats.
In the end, the nested chat handler uses the results of the nested chats
to produce a response to the original message.
By default, the nested chat handler uses the summary of the last chat
as the response.

Here is an example of using nested chats to build an arithmetic agent that
packages arithmetic operations, code-based validation, and poetry into a single
agent. This arithmetic agent takes a number transformation request like 
"turn number 3 into 13" and returns a poem that describes a transformation attempt.

First we define the agents. We reuse the `group_chat_manager_with_intros`
from previous example to orchestrate the arithmetic operations.

In [31]:
import tempfile

temp_dir = tempfile.gettempdir()

arithmetic_agent = ConversableAgent(
    name="Arithmetic_Agent",
    llm_config=False,
    human_input_mode="ALWAYS",
    # This agent will always require human input to make sure the code is
    # safe to execute.
    code_execution_config={"use_docker": False, "work_dir": temp_dir},
)

code_writer_agent = ConversableAgent(
    name="Code_Writer_Agent",
    system_message="You are a code writer. You write Python script in Markdown code blocks.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

poetry_agent = ConversableAgent(
    name="Poetry_Agent",
    system_message="You are an AI poet.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

Now we define the nested chats using the sequential chat pattern.
All the senders are always `artihmetic_agent`.

In [28]:
nested_chats = [
    {
        "recipient": group_chat_manager_with_intros,
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Summarize the sequence of operations used to turn " "the source number into target number.",
    },
    {
        "recipient": code_writer_agent,
        "message": "Write a Python script to verify the arithmetic operations is correct.",
        "summary_method": "reflection_with_llm",
    },
    {
        "recipient": poetry_agent,
        "message": "Write a poem about it.",
        "max_turns": 1,
        "summary_method": "last_msg",
    },
]

Now we register the nested chats handler to the `arithmetic_agent` and
set the conditions for triggering the nested chats.

In [ ]:
arithmetic_agent.register_nested_chats(
    nested_chats,
    # The trigger function is used to determine if the agent should start the nested chat
    # given the sender agent.
    # In this case, the arithmetic agent will not start the nested chats if the sender is
    # from the nested chats' recipient to avoid recursive calls.
    trigger=lambda sender: sender not in [group_chat_manager_with_intros, code_writer_agent, poetry_agent],
)

Finally, we call `generate_reply` to get a response from the `arithmetic_agent`
-- this will trigger a sequence of nested chats and return the summary of the
last nested chat as the response.

In [29]:
# Instead of using `initiate_chat` method to start another conversation,
# we can use the `generate_reply` method to get single reply to a message directly.
reply = arithmetic_agent.generate_reply(
    messages=[{"role": "user", "content": "I have a number 3 and I want to turn it into 7."}]
)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

********************************************************************************
Start a new chat with the following message: 
I have a number 3 and I want to turn it into 7.

With the following carryover: 


********************************************************************************
Arithmetic_Agent (to chat_manager):

I have a number 3 and I want to turn it into 7.

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

To give you the result, I'll add 1 to the number you gave me. So your new number is 4.

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

8

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

7

--------------------------------------------------------------------------------
Number_Agent (to chat_manager):

7

A poem is returned as the response, which describes the transformation attempt
from 3 to 7.

The implementation of the nested chats handler makes use of the
[`register_reply`](../reference/agentchat/conversable_agent/#register_reply)
method, which allows you to make extensive customization to
 `ConversableAgent`. The GroupChatManager uses the same mechanism to implement the group chat.

Nested chat is a powerful conversation pattern that allows you to package
complex workflows into a single agent.
You can hide [tool usages](/docs/tutorial/tool-use) within a single agent by having the tool-caller agent 
starts a nested chat with a tool-executor agent and then use the result
of the nested chat to generate a response.
See the [nested chats for tool use notebook](/docs/notebooks/agentchat_nested_chats_chess) for an example.

## Customzing Speaker Selection

In GroupChat, we can also customize the speaker selection by passing in a function to `speaker_selection_method`:
```python
def custom_speaker_selection_func(last_speaker, groupchat):
    """Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.

    Parameters:
        - last_speaker: Agent
            The last speaker in the group chat.
        - groupchat: GroupChat
            The GroupChat object
    Return:
        Return one of the following:
        1. an `Agent` class, it must be one of the agents in the group chat.
        2. a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
        3. None, which indicates the chat should be terminated.
    """
    pass

groupchat = autogen.GroupChat(
    speaker_selection_method=custom_speaker_selection_func,
    ...,
)
```
The last speaker and the groupchat object are passed to the function. 
Commonly used variables from groupchat are `groupchat.messages` and `groupchat.agents`, which is the message history and the agents in the group chat respectively. You can access other attributes of the groupchat, such as `groupchat.allowed_speaker_transitions_dict` for pre-defined `allowed_speaker_transitions_dict`.

Heres is a simple example to build workflow for reasearch with customized speaker selection.


```{=mdx}
![group_chat](../../blog/2024-02-29-StateFlow/img/sf_example_1.png)
```

We define the following agents:

- Initializer: Start the workflow by sending a task.
- Coder: Retrieve papers from the internet by writing code.
- Executor: Execute the code.
- Scientist: Read the papers and write a summary.

In the Figure, we define a simple workflow for research with 4 states: Init, Retrieve, Reserach and End. Within each state, we will call different agents to perform the tasks.

Init: We use the initializer to start the workflow.
Retrieve: We will first call the coder to write code and then call the executor to execute the code.
Research: We will call the scientist to read the papers and write a summary.
End: We will end the workflow.

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-1106-preview"],
    },
)

In [ ]:
gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

initializer = autogen.UserProxyAgent(
    name="Init",
)

coder = autogen.AssistantAgent(
    name="Retrieve_Action_1",
    llm_config=gpt4_config,
    system_message="""You are the Coder. Given a topic, write code to retrieve related papers from the arXiv API, print their title, authors, abstract, and link.
You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)
executor = autogen.UserProxyAgent(
    name="Retrieve_Action_2",
    system_message="Executor. Execute the code written by the Coder and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
scientist = autogen.AssistantAgent(
    name="Research_Action_1",
    llm_config=gpt4_config,
    system_message="""You are the Scientist. Please categorize papers after seeing their abstracts printed and create a markdown table with Domain, Title, Authors, Summary and Link""",
)


def state_transition(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is initializer:
        # init -> retrieve
        return coder
    elif last_speaker is coder:
        # retrieve: action 1 -> action 2
        return executor
    elif last_speaker is executor:
        if messages[-1]["content"] == "exitcode: 1":
            # retrieve --(execution failed)--> retrieve
            return coder
        else:
            # retrieve --(execution sucess)--> research
            return scientist
    elif last_speaker == "Scientist":
        # research -> end
        return None


groupchat = autogen.GroupChat(
    agents=[initializer, coder, executor, scientist],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

In [ ]:
initializer.initiate_chat(
    manager, message="Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains."
)

Init (to chat_manager):

Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.

--------------------------------------------------------------------------------
Retrieve_Action_1 (to chat_manager):

To retrieve related papers from the arXiv API, we can use Python with the `requests` library to send a query to the API and parse the response. Below is a Python script that searches for papers related to "LLM applications" (Large Language Models applications) from the last week, across different domains, and prints out the required information for 5 to 10 papers.

```python
import requests
from datetime import datetime, timedelta
import feedparser

# Define the base URL for the arXiv API
ARXIV_API_URL = 'http://export.arxiv.org/api/query?'

# Define the search parameters
search_query = 'all:"LLM applications"'
start_date = (datetime.now() - timedelta(days=7)).strftime('%Y%m%d%H%M%S')
end_date = datetime.now().strftime('%Y%m%d%H%M%S')
start = 0
ma

ChatResult(chat_id=None, chat_history=[{'content': 'Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.', 'role': 'assistant'}, {'content': 'To retrieve related papers from the arXiv API, we can use Python with the `requests` library to send a query to the API and parse the response. Below is a Python script that searches for papers related to "LLM applications" (Large Language Models applications) from the last week, across different domains, and prints out the required information for 5 to 10 papers.\n\n```python\nimport requests\nfrom datetime import datetime, timedelta\nimport feedparser\n\n# Define the base URL for the arXiv API\nARXIV_API_URL = \'http://export.arxiv.org/api/query?\'\n\n# Define the search parameters\nsearch_query = \'all:"LLM applications"\'\nstart_date = (datetime.now() - timedelta(days=7)).strftime(\'%Y%m%d%H%M%S\')\nend_date = datetime.now().strftime(\'%Y%m%d%H%M%S\')\nstart = 0\nmax_results = 10\nsort_by = \'submi

## Summary

In this chapter, we covered two-agent chat, sequential chat, group chat,
and nested chat patterns. You can compose these patterns like LEGO blocks to 
create complex workflows. You can also use [`register_reply`](../reference/agentchat/conversable_agent/#register_reply) to create new patterns.

This is the last chapter on basic AutoGen concepts. 
In the next chatper, we will give you some tips on what to do next.